# 04: Optional Aer GPU Smoke Test

This notebook checks for optional Qiskit Aer GPU support and runs a tiny GPU simulation when available.

If GPU support is unavailable on the current machine, it prints a clear skip message and exits cleanly.

In [ ]:
import qiskit
import qiskit_aer
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2

print('Qiskit version:', qiskit.__version__)
print('qiskit-aer version:', qiskit_aer.__version__)

In [ ]:
base_sim = AerSimulator()

try:
    available_devices = tuple(base_sim.available_devices())
except Exception as exc:
    available_devices = ('CPU',)
    print('Could not query available devices; assuming CPU-only:', repr(exc))

print('Aer available devices:', available_devices)

In [ ]:
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

cpu_sim = AerSimulator(device='CPU')
cpu_pm = generate_preset_pass_manager(backend=cpu_sim, optimization_level=1)
isa_cpu_qc = cpu_pm.run(qc)
cpu_sampler = SamplerV2.from_backend(cpu_sim)
cpu_result = cpu_sampler.run([isa_cpu_qc], shots=512).result()[0]
cpu_meas = cpu_result.data["meas"]
cpu_counts = cpu_meas.get_counts()
print('CPU counts:', cpu_counts)

In [ ]:
if 'GPU' not in available_devices:
    print('GPU support not detected on this machine. Skipping GPU smoke test.')
else:
    try:
        gpu_sim = AerSimulator(method='statevector', device='GPU')
        gpu_pm = generate_preset_pass_manager(backend=gpu_sim, optimization_level=1)
        isa_gpu_qc = gpu_pm.run(qc)

        gpu_sampler = SamplerV2.from_backend(gpu_sim)
        gpu_result = gpu_sampler.run([isa_gpu_qc], shots=512).result()[0]
        gpu_meas = gpu_result.data["meas"]
        gpu_counts = gpu_meas.get_counts()

        print('GPU counts:', gpu_counts)
    except Exception as exc:
        print('GPU execution path failed:', repr(exc))